<a href="https://colab.research.google.com/github/sgu20191816/krx_competition/blob/main/krx_%EC%A3%BC%EC%8B%9D%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import

In [ ]:
!pip install pykrx --quiet
from pykrx import stock
from pykrx import bond

!pip install prophet --quiet
from prophet import Prophet

!pip install workalendar --quiet
from workalendar.asia import SouthKorea

import time
import math
from tqdm import tqdm
import pandas as pd
import numpy as np

In [ ]:
# 예측에 필요한 주식 종목들 확인을 위해 데이콘 제공 csv파일을 불러옴
init1 = pd.read_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/open/train.csv')
init2 = pd.read_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/open/train_additional.csv')

# Data Preprocessing

In [ ]:
# 'A'를 제외하고 고유한 종목코드를 추출
ticker_list = init1['종목코드'].str[1:].unique()
ticker_list = ticker_list.tolist()

#7월 중에 상장 폐지된 종목은 제거
ticker_list.remove('096640')

len(ticker_list)

1999

- 기업의 행위(액면 변경, 배당 등)에 의한 주가의 변동을 고려한 수정 주가를 활용하고자 함
- 공공데이터로 개방된 KRX, NAVER에서 수정 주가 및 시가총액을 불러올 수 있는 크롤링 라이브러리 pykrx를 활용

In [ ]:
# pykrx 라이브러리를 사용해 "20220601"~"20230728" 데이터를 train에 저장
train = dict()

for ticker in tqdm(ticker_list) :
   train[ticker] = stock.get_market_ohlcv("20220601", "20230728", ticker)
   train[ticker].to_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/{}.csv'.format(ticker), index=False)

In [ ]:
init = pd.concat([init1, init2])

In [ ]:
dates = init['일자'].unique()

# 필터링할 날짜 범위
start_date = 20220601
end_date = 20230728

# 범위 내의 날짜만 선택
filtered_dates = [date for date in dates if start_date <= date <= end_date]

In [ ]:
# 사용 모델에서 휴일 고려를 위해, 한국 기준 휴일 추출
date_list = pd.date_range(start='20220601', end='20230821', freq='D')
kr_holidays = stock.get_previous_business_days(fromdate = '20220601', todate = '20230821')
holiday_df = pd.DataFrame(columns=['ds','holiday'])
holiday_df['ds'] = sorted(date_list)
holiday_df['holiday'] = holiday_df.ds.apply(lambda x: 'non-holiday' if x in kr_holidays else 'holiday')

In [ ]:
def create_empty_df(columns, index):
    return pd.DataFrame(0, index=index, columns=columns)

merged_data = dict()
for ticker in tqdm(ticker_list):
  if ticker == '096640': continue # 상장폐지된 종목 pass
  train[ticker] = pd.read_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/{}.csv'.format(ticker))
  train[ticker]['일자'] = filtered_dates

  train[ticker]['일자'] = pd.to_datetime(train[ticker]['일자'], format='%Y%m%d').dt.date
  train[ticker] = train[ticker].set_index('일자')
  merged_data[ticker] = train[ticker].fillna(0)

100%|██████████| 1999/1999 [05:26<00:00,  6.11it/s]


In [ ]:
# 일간 수익률 = (현시점 거래일의 종가 - 전날 거래일의 종가) / 전날거래일의 종가
# 연율화 = 일간 * 거래 기간
# n일차 일간 수익률 = 연율화된 일간 수익률

# 변동성을 계산하기 위해 총 일수 확인
일수 = len(merged_data[ticker_list[0]])

for ticker in tqdm(ticker_list) :
  if ticker == '096640': continue
  merged_data[ticker]['연율화'] = 0
  merged_data[ticker]['일자'] = merged_data[ticker].index
  merged_data[ticker] = merged_data[ticker].reset_index(drop=True)
  merged_data[ticker]['연율화'] = 0

  # n+1번째 날 일간 수익률
  for n in range(1, len(merged_data[ticker])) :
    전날종가 = merged_data[ticker].iloc[n-1]['종가']
    현재종가 = merged_data[ticker].iloc[n]['종가']

    n일간수익률 = (현재종가 - 전날종가) / 전날종가
    n연율화 = n일간수익률 * len(merged_data[ticker])
    merged_data[ticker].loc[n, '연율화'] = n연율화

100%|██████████| 1999/1999 [03:41<00:00,  9.03it/s]


In [ ]:
merged_data[ticker]

,시가,고가,저가,종가,거래량,거래대금,등락률,연율화,일자
0,8050,8120,7960,8020,4450,35652240,-0.37,0.000000,2022-06-02
1,8030,8120,8010,8070,6293,50668960,0.62,1.795511,2022-06-03
2,7990,8070,7990,8000,2641,21203260,-0.87,-2.498141,2022-06-07
3,7950,8090,7950,7990,4818,38598630,-0.13,-0.360000,2022-06-08
4,8000,8000,7800,7900,9167,72208510,-1.13,-3.244055,2022-06-09
...,...,...,...,...,...,...,...,...,...
283,7040,7080,6720,6830,73790,505764830,-3.53,-10.169492,2023-07-24
284,6760,6870,6500,6530,51301,340374190,-4.39,-12.650073,2023-07-25
285,6420,6590,6200,6300,128660,816159440,-3.52,-10.143951,2023-07-26
286,6140,6620,6120,6460,42368,272530940,2.54,7.314286,2023-07-27


- 포트폴리오의 안정성을 위해 시가총액과 주가의 변동성의 표준편차를 함께 고려하여 정렬

In [ ]:
# 단위가 다른 시가총액과 변동성 value - scaling
def minmax_scale_dict_values(input_dict):
    values = list(input_dict.values())
    min_value = min(values)
    max_value = max(values)

    scaled_dict = {}
    for key, value in input_dict.items():
        scaled_value = (value - min_value) / (max_value - min_value)
        scaled_dict[key] = scaled_value

    return scaled_dict

In [ ]:
# 시가총액은 클수록, 변동성은 작을수록 선호 - 비례 관계 맞춰줌
def subtract_one_from_dict_values(input_dict):
    subtracted_dict = {}
    for key, value in input_dict.items():
        if ( value == 0 ) : continue
        subtracted_dict[key] = 1- value

    return subtracted_dict

In [ ]:
# 각 종목당 편차 계산
def exponential_smoothing(alpha, data):
    forecast = [data[0]] # 초기 예측값

    for i in range(1, len(data)):
        smoothed_value = alpha * data[i] + (1 - alpha) * forecast[i - 1]
        forecast.append(smoothed_value)

    return forecast

alpha = 0.5

표준편차 = dict()

for ticker in ticker_list :
  sum = 0
  merged_data[ticker]['ewma'] = exponential_smoothing(alpha, merged_data[ticker]['연율화'])
  연율화평균 = merged_data[ticker]['ewma'].mean()

  for n in range(1,len(merged_data[ticker])) :
    sum += (merged_data[ticker]['ewma'][n] - 연율화평균)**2

  변동성 = (sum/(len(merged_data[ticker])-2))**(0.5)
  표준편차[ticker] = 변동성

scaled_std = minmax_scale_dict_values(표준편차)
scaled_std = subtract_one_from_dict_values(scaled_std)

In [ ]:
# 가장 최근의 시총 불러와서 저장
market_cap = dict()
for ticker in tqdm(ticker_list) :
   temp= stock.get_market_cap("20230728", "20230728", ticker)
   market_cap[ticker] = temp['시가총액']

market_cap = pd.DataFrame.from_dict(market_cap, orient='index').reset_index()
market_cap.columns = ['종목코드', '시총']
market_cap = market_cap.sort_values(by='시총', ascending=False)

market_cap.to_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/시가총액.csv',index=False)

# 저장한 시총 읽어들이기
market_cap = pd.read_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/시가총액.csv')

recode = []
for ticker in market_cap['종목코드'] :
  code = str(ticker).zfill(6)
  recode.append(code)

market_cap['종목코드'] = recode

In [ ]:
# 각 종목 당 시총 저장
rank_dict = dict()

for ticker in ticker_list :
  if ( 표준편차[ticker] == 0 ) : continue
  rank = market_cap.loc[market_cap['종목코드'] == ticker, '시총'].iloc[0]
  rank_dict[ticker] = rank

scaled_rank = minmax_scale_dict_values(rank_dict)

In [ ]:
# scaled 시총 + 변동성
std_rank = dict()

for ticker in ticker_list :
  if ( 표준편차[ticker] == 0 ) : continue
  s = scaled_std[ticker]
  r = scaled_rank[ticker]
  std_rank[ticker] = (s+r)

In [ ]:
정렬 = dict(sorted(std_rank.items(), key = lambda x : x[1], reverse=True))
정렬 = pd.DataFrame.from_dict(정렬, orient='index').reset_index()
정렬.columns = ['종목코드', 'std_rank']
정렬.head()

- 종목의 long과 short을 선별하는 기준점을 잡기 위해 한달동안의 상승 종목과 하락 종목의 평균을 구함

In [ ]:
pos_fr = []
neg_fr = []
for ticker in tqdm(ticker_list) :
  df = merged_data[ticker]
  one_month = df.iloc[-15:]
  means = round(one_month['등락률'].mean(), 5)

  if means > 0:
    pos_fr.append(means)
  else:
    neg_fr.append(means)

print()
pos_fr = np.array(pos_fr)
neg_fr = np.array(neg_fr)
print('1달동안 주가가 상승한 경우의 평균 상승률:', np.median(pos_fr))
print('1달동안 주가가 하락한 경우의 평균 하락률:', np.median(neg_fr))

#1달동안 주가가 상승한 경우의 평균 상승률: 0.44833500000000004
#1달동안 주가가 하락한 경우의 평균 하락률: -0.44333

# Model Train & Predict

In [ ]:
# 기간의 시작일의 시가와 기간의 마지막일의 종가를 비교
# 양수는 long , 음수는 short
import logging
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)
logging.getLogger('prophet').setLevel(logging.WARNING)

import matplotlib.pyplot as plt
import numpy as np
import time

forecast = dict()

long = []
short = []
#prophet모델의 예측 등락률이 낮아서 반영비율(weight) 적용
w = 4

def calculate_final_value(start_value, 등락률):
    final_value = start_value
    for rate in 등락률:
        final_value *= (1 + (rate/100)*w)
    return final_value

for i in tqdm(range(2000)) :
  #if (정렬.loc[i, '표준편차'] == 0 ) : continue # 거래정지, 상장폐지 등의 이유로 예상되어 패스함
  ticker = 정렬.loc[i, '종목코드'] # 종목코드 선택

  if len(long) >= 200 and len(short) >= 300: # 조기 종료조건
      break

  df = merged_data[ticker].rename(columns={'일자': 'ds', '등락률': 'y'}) # 순차적으로 데이터프레임 선택
  df['ds'] = pd.to_datetime(df['ds'], format='%Y-%m-%d')
  df = df.fillna(0)

  model = Prophet(holidays=holiday_df, changepoint_prior_scale=0.15, daily_seasonality = True, seasonality_mode = 'additive', seasonality_prior_scale = 10)
  model.fit(df)

  future = model.make_future_dataframe(periods=22, freq = 'D')
  forecast[ticker] = model.predict(future)

  yhat = calculate_final_value(df['종가'].iloc[-1],forecast[ticker]['yhat'].iloc[-22:])

  if yhat >= (1.045*df['종가'].iloc[-1]) :
      long.append(ticker)
  elif yhat <= (0.956*df['종가'].iloc[-1]):
      short.append(ticker)

In [ ]:
#short에 있는 선별된 종목들 중에서 공매도 비중이 높은 순으로 최종 200주 결정
숏공매도 = dict()
for ticker in tqdm(short) :
   df = stock.get_shorting_volume_by_date("20230728", "20230728", ticker)
   숏공매도[ticker] = df['비중']

숏공매도비중 = pd.DataFrame.from_dict(숏공매도, orient='index').reset_index()
숏공매도비중.columns = ['종목코드', '비중']
숏공매도비중 = 숏공매도비중.sort_values(by='비중', ascending=False)
숏공매도비중[:200]

In [ ]:
정답롱 = long[:200]
정답숏 = 숏공매도비중['종목코드'][:200]

ticker_list = init['종목코드'].str[1:].unique()

정답롱 = ['A' + item for item in 정답롱]
정답숏 = ['A' + item for item in 정답숏]
ticker_list = ['A' + item for item in ticker_list]
나머지 = [x for x in ticker_list if x not in (정답롱 + 정답숏)]

# Submission

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/open/sample_submission.csv')

submission.loc[:199, '종목코드'] = 정답롱
submission.loc[200:1799, '종목코드'] = 나머지
submission.loc[1800:, '종목코드'] = 정답숏

In [ ]:
submission.to_csv('/content/drive/MyDrive/DACON/KRX 주식 투자 알고리즘 경진대회/open/sub_final.csv')